In [1]:
#!/usr/bin/python3.6
import random as r
import json

from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from AbstractArchitecture_v2 import AbstractArchitecture
from DenseEncoder import DenseEncoder
from DenseDecoder import DenseDecoder
from NormalizedMeanSquaredError import NormalizedMeanSquaredError as NMSE



# Set Experiment Specifics
expt_name = "CheckpointTest-a"
data_file_prefix = './data/Duffing_Equation_expt4'  ## FILL IN HERE (from file name)

# Network architecture design
l = 20  # Latent space size

activation = "relu"
initializer = tf.keras.initializers.VarianceScaling()
reg_lambda_l2 = 1e-6
regularizer = tf.keras.regularizers.l2(reg_lambda_l2)

act_layer = dict(activation=activation,
                 kernel_initializer=initializer,
                 kernel_regularizer=regularizer)
lin_layer = dict(activation=None,
                 kernel_initializer=initializer,
                 kernel_regularizer=regularizer)
latent_config = dict(activation=None,
                     kernel_regularizer=regularizer,
                     use_bias=False)

encoder_layers = 5
decoder_layers = 5
add_identity = True

# Model training setting
## Set optimizer
optimizer = keras.optimizers.Adam
optimizer_opts = {}

# Callback function(s) and fit method options
#cbs = [keras.callbacks.EarlyStopping(patience=10)]
cbs = [keras.callbacks.ModelCheckpoint('CheckpointTest.tmp',
                                       save_weights_only=True,
                                       monitor='val_loss',
                                       save_best_only=True)]

# Batch size for model training
batch_size = 64

# Time to train autoencoders only and full models for initial seeding test
num_init_models = 4  # number of models to try with diff. learning rates
aec_only_time = 2  # minutes
full_model_time = 10  # minutes

# This number is used to compute number of epochs for full-model training
final_model_train_hrs = 0.5


############################################
### Everything below here is automated!! ###
############################################

# Step 0. Assign a random number generator seed
x = r.randint(0, 10**(10))
r.seed(x)

# Step 1. Load in the data
data_train_u = np.load("{}_train1_u.npy".format(data_file_prefix))
data_train_f = np.load("{}_train1_f.npy".format(data_file_prefix))
data_val_u = np.load("{}_val_u.npy".format(data_file_prefix))
data_val_f = np.load("{}_val_f.npy".format(data_file_prefix))
data_test_u1 = np.load("{}_test1_u.npy".format(data_file_prefix))
data_test_f1 = np.load("{}_test1_f.npy".format(data_file_prefix))
data_test_u = np.load("{}_test2_u.npy".format(data_file_prefix))
data_test_f = np.load("{}_test2_f.npy".format(data_file_prefix))

# Step 2. Set up the model architecture
_, n = data_train_u.shape

encoder_config = {'units_full': n,
                  'num_layers': encoder_layers,
                  'actlay_config': act_layer,
                  'linlay_config': lin_layer,
                  'add_init_fin': add_identity}

decoder_config = {'units_full': n,
                  'num_layers': decoder_layers,
                  'actlay_config': act_layer,
                  'linlay_config': lin_layer,
                  'add_init_fin': add_identity}

# Aggregate settings for model architecture
architecture_config = {'units_latent': l,
                       'units_full': n,
                       'u_encoder_block': DenseEncoder(**encoder_config),
                       'u_decoder_block': DenseDecoder(**decoder_config),
                       'F_encoder_block': DenseEncoder(**encoder_config),
                       'F_decoder_block': DenseDecoder(**decoder_config),
                       'latent_config': latent_config}

###############################################
## Step 3. Train 20 initial models, autoencoders-only then full model
###############################################
# create a variety of different models with randomized learning rates
models = []

# Set the loss functions
loss_fns = 4*[NMSE()]

# Set up validation data for autoencoders-only and full model training
val_zeros = np.zeros(data_val_u.shape)
val_data_aec = [(data_val_u, data_val_f), 
                (data_val_u, data_val_f, val_zeros, val_zeros)]
val_data_full = [(data_val_u, data_val_f), 
                 (data_val_u, data_val_f, data_val_f, data_val_u)]

# Compute number of epochs to train
aec_epochs = int(aec_only_time*60*2)  # about 2 epochs/sec
full_epochs = int(full_model_time*60)  # about 1 epoch/sec


# For loop for generating the different models
for i in range(num_init_models):
    # Randomly selected learning rate
    lr = 10**(-r.uniform(3, 6))
    
    # Create a model, initially only to train autoencoders!
    model = AbstractArchitecture(**architecture_config,
                                 train_autoencoders_only=True)
    # Compile the model
    model.compile(loss=loss_fns,
                  optimizer=optimizer(learning_rate=lr, **optimizer_opts))

    # Fit the model
    train_zeros = np.zeros(data_train_u.shape)
    aec_hist = model.fit(x=[data_train_u, data_train_f],
                         y=[data_train_u, data_train_f, train_zeros, train_zeros],
                         validation_data=val_data_aec,
                         #callbacks=cbs,
                         batch_size=batch_size,
                         epochs=aec_epochs)
    
    # Now set the model to train all aspects (including operator L)
    model.train_autoencoders_only = False

    # Re-compile the model
    model.compile(loss=loss_fns,
                  optimizer=optimizer(learning_rate=lr, **optimizer_opts))

    # Train full model
    hist = model.fit(x=[data_train_u, data_train_f],
                     y=[data_train_u, data_train_f, data_train_f, data_train_u],
                     validation_data=val_data_full,
                     #callbacks=cbs,
                     batch_size=batch_size,
                     epochs=full_epochs)
    
    # Append the results to the model list
    models.append((model, hist, lr, aec_hist))

Train on 4798 samples, validate on 1200 samples
Epoch 1/240
4798/4798 [==============================] - 2s 424us/sample - loss: 2.0130 - output_1_loss: 0.9995 - output_2_loss: 1.0107 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 2.0094 - val_output_1_loss: 0.9992 - val_output_2_loss: 1.0074 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 2/240
4798/4798 [==============================] - 0s 83us/sample - loss: 2.0069 - output_1_loss: 0.9984 - output_2_loss: 1.0057 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 2.0036 - val_output_1_loss: 0.9981 - val_output_2_loss: 1.0027 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 3/240
4798/4798 [==============================] - 0s 77us/sample - loss: 2.0011 - output_1_loss: 0.9973 - output_2_loss: 1.0010 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.9981 - val_output_1_loss: 0.9970 - val_output_2_loss: 0.9983 - val_output_3_loss: 0.00

Epoch 25/240
4798/4798 [==============================] - 0s 75us/sample - loss: 1.6757 - output_1_loss: 0.8758 - output_2_loss: 0.7971 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.6685 - val_output_1_loss: 0.8724 - val_output_2_loss: 0.7930 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 26/240
4798/4798 [==============================] - 0s 74us/sample - loss: 1.6505 - output_1_loss: 0.8651 - output_2_loss: 0.7826 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.6441 - val_output_1_loss: 0.8618 - val_output_2_loss: 0.7793 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 27/240
4798/4798 [==============================] - 0s 74us/sample - loss: 1.6264 - output_1_loss: 0.8547 - output_2_loss: 0.7688 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.6208 - val_output_1_loss: 0.8516 - val_output_2_loss: 0.7661 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 2

Epoch 50/240
4798/4798 [==============================] - 0s 74us/sample - loss: 1.3208 - output_1_loss: 0.6955 - output_2_loss: 0.6224 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.3171 - val_output_1_loss: 0.6872 - val_output_2_loss: 0.6267 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 51/240
4798/4798 [==============================] - 0s 75us/sample - loss: 1.3127 - output_1_loss: 0.6897 - output_2_loss: 0.6202 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.3087 - val_output_1_loss: 0.6811 - val_output_2_loss: 0.6245 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 52/240
4798/4798 [==============================] - 0s 76us/sample - loss: 1.3048 - output_1_loss: 0.6839 - output_2_loss: 0.6180 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.3005 - val_output_1_loss: 0.6749 - val_output_2_loss: 0.6224 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 5

Epoch 75/240
4798/4798 [==============================] - 0s 74us/sample - loss: 1.1458 - output_1_loss: 0.5504 - output_2_loss: 0.5925 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.1342 - val_output_1_loss: 0.5339 - val_output_2_loss: 0.5973 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 76/240
4798/4798 [==============================] - 0s 75us/sample - loss: 1.1393 - output_1_loss: 0.5446 - output_2_loss: 0.5919 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.1275 - val_output_1_loss: 0.5278 - val_output_2_loss: 0.5966 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 77/240
4798/4798 [==============================] - 0s 74us/sample - loss: 1.1328 - output_1_loss: 0.5386 - output_2_loss: 0.5913 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 1.1207 - val_output_1_loss: 0.5216 - val_output_2_loss: 0.5960 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 7

Epoch 100/240
4798/4798 [==============================] - 0s 75us/sample - loss: 1.0022 - output_1_loss: 0.4230 - output_2_loss: 0.5765 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 0.9878 - val_output_1_loss: 0.4046 - val_output_2_loss: 0.5802 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 101/240
4798/4798 [==============================] - 0s 75us/sample - loss: 0.9977 - output_1_loss: 0.4194 - output_2_loss: 0.5756 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 0.9832 - val_output_1_loss: 0.4010 - val_output_2_loss: 0.5792 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoch 102/240
4798/4798 [==============================] - 0s 73us/sample - loss: 0.9933 - output_1_loss: 0.4160 - output_2_loss: 0.5745 - output_3_loss: 0.0000e+00 - output_4_loss: 0.0000e+00 - val_loss: 0.9786 - val_output_1_loss: 0.3975 - val_output_2_loss: 0.5781 - val_output_3_loss: 0.0000e+00 - val_output_4_loss: 0.0000e+00
Epoc

...

Epoch 478/600
4798/4798 [==============================] - 0s 85us/sample - loss: 1.7832 - output_1_loss: 0.0177 - output_2_loss: 0.5178 - output_3_loss: 1.0829 - output_4_loss: 0.0323 - val_loss: 1.6610 - val_output_1_loss: 0.0171 - val_output_2_loss: 0.5101 - val_output_3_loss: 0.9236 - val_output_4_loss: 0.0788
Epoch 479/600
4798/4798 [==============================] - 0s 84us/sample - loss: 32.9845 - output_1_loss: 0.0453 - output_2_loss: 0.8059 - output_3_loss: 31.8729 - output_4_loss: 0.0369 - val_loss: 1.9629 - val_output_1_loss: 0.0232 - val_output_2_loss: 0.6725 - val_output_3_loss: 1.0167 - val_output_4_loss: 0.0629
Epoch 480/600
4798/4798 [==============================] - 0s 84us/sample - loss: 1.8677 - output_1_loss: 0.0177 - output_2_loss: 0.6022 - output_3_loss: 1.0641 - output_4_loss: 0.0276 - val_loss: 1.6503 - val_output_1_loss: 0.0165 - val_output_2_loss: 0.5905 - val_output_3_loss: 0.8432 - val_output_4_loss: 0.0560
Epoch 481/600
4798/4798 [=========================

Epoch 504/600
4798/4798 [==============================] - 0s 86us/sample - loss: 1.0164 - output_1_loss: 0.0062 - output_2_loss: 0.4376 - output_3_loss: 0.5052 - output_4_loss: 0.0177 - val_loss: 1.1071 - val_output_1_loss: 0.0078 - val_output_2_loss: 0.4575 - val_output_3_loss: 0.5252 - val_output_4_loss: 0.0677
Epoch 505/600
4798/4798 [==============================] - 0s 85us/sample - loss: 1.0116 - output_1_loss: 0.0063 - output_2_loss: 0.4362 - output_3_loss: 0.4993 - output_4_loss: 0.0193 - val_loss: 1.1145 - val_output_1_loss: 0.0085 - val_output_2_loss: 0.4561 - val_output_3_loss: 0.5263 - val_output_4_loss: 0.0772
Epoch 506/600
4798/4798 [==============================] - 0s 86us/sample - loss: 1.0035 - output_1_loss: 0.0061 - output_2_loss: 0.4352 - output_3_loss: 0.4962 - output_4_loss: 0.0182 - val_loss: 1.0856 - val_output_1_loss: 0.0083 - val_output_2_loss: 0.4555 - val_output_3_loss: 0.5186 - val_output_4_loss: 0.0568
Epoch 507/600
4798/4798 [===========================

Epoch 530/600
4798/4798 [==============================] - 0s 84us/sample - loss: 0.8423 - output_1_loss: 0.0058 - output_2_loss: 0.3604 - output_3_loss: 0.4131 - output_4_loss: 0.0194 - val_loss: 0.9144 - val_output_1_loss: 0.0080 - val_output_2_loss: 0.3767 - val_output_3_loss: 0.4307 - val_output_4_loss: 0.0572
Epoch 531/600
4798/4798 [==============================] - 0s 84us/sample - loss: 0.8218 - output_1_loss: 0.0055 - output_2_loss: 0.3549 - output_3_loss: 0.4027 - output_4_loss: 0.0179 - val_loss: 0.9181 - val_output_1_loss: 0.0079 - val_output_2_loss: 0.3714 - val_output_3_loss: 0.4366 - val_output_4_loss: 0.0604
Epoch 532/600
4798/4798 [==============================] - 0s 83us/sample - loss: 0.8732 - output_1_loss: 0.0069 - output_2_loss: 0.3560 - output_3_loss: 0.4312 - output_4_loss: 0.0179 - val_loss: 1.2127 - val_output_1_loss: 0.0101 - val_output_2_loss: 0.3944 - val_output_3_loss: 0.6504 - val_output_4_loss: 0.0613
Epoch 533/600
4798/4798 [===========================

Epoch 556/600
4798/4798 [==============================] - 0s 85us/sample - loss: 1.1742 - output_1_loss: 0.0079 - output_2_loss: 0.4816 - output_3_loss: 0.5939 - output_4_loss: 0.0202 - val_loss: 1.2842 - val_output_1_loss: 0.0105 - val_output_2_loss: 0.5095 - val_output_3_loss: 0.6397 - val_output_4_loss: 0.0581
Epoch 557/600
4798/4798 [==============================] - 0s 84us/sample - loss: 1.1500 - output_1_loss: 0.0074 - output_2_loss: 0.4770 - output_3_loss: 0.5801 - output_4_loss: 0.0179 - val_loss: 1.2512 - val_output_1_loss: 0.0092 - val_output_2_loss: 0.4916 - val_output_3_loss: 0.6345 - val_output_4_loss: 0.0483
Epoch 558/600
4798/4798 [==============================] - 0s 84us/sample - loss: 1.1301 - output_1_loss: 0.0073 - output_2_loss: 0.4713 - output_3_loss: 0.5680 - output_4_loss: 0.0176 - val_loss: 1.2219 - val_output_1_loss: 0.0097 - val_output_2_loss: 0.4876 - val_output_3_loss: 0.6093 - val_output_4_loss: 0.0528
Epoch 559/600
4798/4798 [===========================

Epoch 582/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.8697 - output_1_loss: 0.0050 - output_2_loss: 0.3828 - output_3_loss: 0.4233 - output_4_loss: 0.0176 - val_loss: 0.9714 - val_output_1_loss: 0.0075 - val_output_2_loss: 0.3993 - val_output_3_loss: 0.4470 - val_output_4_loss: 0.0767
Epoch 583/600
4798/4798 [==============================] - 0s 84us/sample - loss: 0.8695 - output_1_loss: 0.0055 - output_2_loss: 0.3811 - output_3_loss: 0.4222 - output_4_loss: 0.0202 - val_loss: 0.9511 - val_output_1_loss: 0.0073 - val_output_2_loss: 0.3962 - val_output_3_loss: 0.4375 - val_output_4_loss: 0.0708
Epoch 584/600
4798/4798 [==============================] - 0s 85us/sample - loss: 0.8575 - output_1_loss: 0.0046 - output_2_loss: 0.3789 - output_3_loss: 0.4184 - output_4_loss: 0.0159 - val_loss: 0.9532 - val_output_1_loss: 0.0081 - val_output_2_loss: 0.3946 - val_output_3_loss: 0.4372 - val_output_4_loss: 0.0733
Epoch 585/600
4798/4798 [===========================

In [2]:
###############################################
## Step 4. Select the best model from the 20 autoencoder-only results
###############################################

# List of learning rates and final losses, losses averaged over final 5 epochs
lrs = []
final_losses = []

for i in range(num_init_models):
    _, hist, lr, _ = models[i]
    final_losses.append(np.mean(hist.history['loss'][-5:]))
    lrs.append(lr)


# Select the best model, based on the minimum in the final losses
best_model_idc = np.argmin(final_losses)
best_model = models[best_model_idc][0]
best_lr = lrs[best_model_idc]

# Save weights for the best model
model_weight_path = "./model_weights/{}_best_aec_model_weights.tf".format(expt_name)
best_model.save_weights(model_weight_path)

In [3]:
###############################################
# Step 5. Set up the full architecture run!!
###############################################

# Set up validation data, loss functions, and number of epochs
val_data = [(data_val_u, data_val_f), 
            (data_val_u, data_val_f, data_val_f, data_val_u)]
loss_fns = 4*[NMSE()]

# Compute number of epochs to fit full model
# about 1 epoch/sec in this step
final_epochs = int(final_model_train_hrs*60*60) # 1 epoch/sec

# Instantiate the new model
full_model = AbstractArchitecture(**architecture_config,
                                  train_autoencoders_only=False)

# Load the weights
full_model.load_weights(model_weight_path)
full_model.compile(loss=loss_fns,
                   optimizer=optimizer(learning_rate=best_lr))

# Continue training the (now full) model
hist = full_model.fit(x=[data_train_u, data_train_f],
                      y=[data_train_u, data_train_f, data_train_f, data_train_u],
                      validation_data=val_data,
                      callbacks=cbs,
                      batch_size=batch_size,
                      epochs=full_epochs)

# And save the final results!
full_model_weights_path = "./model_weights/{}_final_model_weights.tf".format(expt_name)
full_model.save_weights(full_model_weights_path)

Train on 4798 samples, validate on 1200 samples
Epoch 1/600
4798/4798 [==============================] - 3s 527us/sample - loss: 0.8465 - output_1_loss: 0.0045 - output_2_loss: 0.3680 - output_3_loss: 0.4199 - output_4_loss: 0.0184 - val_loss: 0.9335 - val_output_1_loss: 0.0071 - val_output_2_loss: 0.3802 - val_output_3_loss: 0.4323 - val_output_4_loss: 0.0725
Epoch 2/600
4798/4798 [==============================] - 0s 100us/sample - loss: 0.8404 - output_1_loss: 0.0045 - output_2_loss: 0.3664 - output_3_loss: 0.4165 - output_4_loss: 0.0157 - val_loss: 1.0313 - val_output_1_loss: 0.0081 - val_output_2_loss: 0.4046 - val_output_3_loss: 0.5159 - val_output_4_loss: 0.0594
Epoch 3/600
4798/4798 [==============================] - 1s 113us/sample - loss: 0.8666 - output_1_loss: 0.0053 - output_2_loss: 0.3736 - output_3_loss: 0.4342 - output_4_loss: 0.0166 - val_loss: 0.9124 - val_output_1_loss: 0.0076 - val_output_2_loss: 0.3721 - val_output_3_loss: 0.4256 - val_output_4_loss: 0.0627
Epoch 4

Epoch 27/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.7988 - output_1_loss: 0.0047 - output_2_loss: 0.3100 - output_3_loss: 0.3744 - output_4_loss: 0.0157 - val_loss: 0.9020 - val_output_1_loss: 0.0075 - val_output_2_loss: 0.3243 - val_output_3_loss: 0.3966 - val_output_4_loss: 0.0666
Epoch 28/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.7645 - output_1_loss: 0.0046 - output_2_loss: 0.3060 - output_3_loss: 0.3644 - output_4_loss: 0.0163 - val_loss: 0.8730 - val_output_1_loss: 0.0075 - val_output_2_loss: 0.3163 - val_output_3_loss: 0.4070 - val_output_4_loss: 0.0588
Epoch 29/600
4798/4798 [==============================] - 0s 87us/sample - loss: 0.7720 - output_1_loss: 0.0051 - output_2_loss: 0.3050 - output_3_loss: 0.3714 - output_4_loss: 0.0191 - val_loss: 0.8514 - val_output_1_loss: 0.0074 - val_output_2_loss: 0.3177 - val_output_3_loss: 0.3965 - val_output_4_loss: 0.0565
Epoch 30/600
4798/4798 [==============================]

Epoch 53/600
4798/4798 [==============================] - 0s 103us/sample - loss: 0.6330 - output_1_loss: 0.0055 - output_2_loss: 0.2517 - output_3_loss: 0.3143 - output_4_loss: 0.0195 - val_loss: 0.6759 - val_output_1_loss: 0.0072 - val_output_2_loss: 0.2535 - val_output_3_loss: 0.3164 - val_output_4_loss: 0.0582
Epoch 54/600
4798/4798 [==============================] - 0s 102us/sample - loss: 0.6155 - output_1_loss: 0.0048 - output_2_loss: 0.2465 - output_3_loss: 0.3078 - output_4_loss: 0.0164 - val_loss: 0.6726 - val_output_1_loss: 0.0072 - val_output_2_loss: 0.2514 - val_output_3_loss: 0.3057 - val_output_4_loss: 0.0607
Epoch 55/600
4798/4798 [==============================] - 0s 90us/sample - loss: 51.4071 - output_1_loss: 0.0070 - output_2_loss: 0.2687 - output_3_loss: 51.0440 - output_4_loss: 0.0175 - val_loss: 25.6927 - val_output_1_loss: 0.0377 - val_output_2_loss: 0.8765 - val_output_3_loss: 24.2107 - val_output_4_loss: 0.0750
Epoch 56/600
4798/4798 [=========================

Epoch 79/600
4798/4798 [==============================] - 0s 87us/sample - loss: 0.8351 - output_1_loss: 0.0055 - output_2_loss: 0.3395 - output_3_loss: 0.4230 - output_4_loss: 0.0157 - val_loss: 0.9032 - val_output_1_loss: 0.0078 - val_output_2_loss: 0.3436 - val_output_3_loss: 0.4437 - val_output_4_loss: 0.0648
Epoch 80/600
4798/4798 [==============================] - 0s 85us/sample - loss: 0.8089 - output_1_loss: 0.0056 - output_2_loss: 0.3376 - output_3_loss: 0.4104 - output_4_loss: 0.0168 - val_loss: 0.9020 - val_output_1_loss: 0.0083 - val_output_2_loss: 0.3395 - val_output_3_loss: 0.4493 - val_output_4_loss: 0.0638
Epoch 81/600
4798/4798 [==============================] - 0s 85us/sample - loss: 0.8035 - output_1_loss: 0.0055 - output_2_loss: 0.3358 - output_3_loss: 0.4109 - output_4_loss: 0.0168 - val_loss: 0.8547 - val_output_1_loss: 0.0070 - val_output_2_loss: 0.3367 - val_output_3_loss: 0.4140 - val_output_4_loss: 0.0661
Epoch 82/600
4798/4798 [==============================]

Epoch 105/600
4798/4798 [==============================] - 0s 84us/sample - loss: 0.7653 - output_1_loss: 0.0049 - output_2_loss: 0.3167 - output_3_loss: 0.3997 - output_4_loss: 0.0159 - val_loss: 0.7920 - val_output_1_loss: 0.0075 - val_output_2_loss: 0.3162 - val_output_3_loss: 0.3790 - val_output_4_loss: 0.0584
Epoch 106/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.7592 - output_1_loss: 0.0047 - output_2_loss: 0.3134 - output_3_loss: 0.3983 - output_4_loss: 0.0154 - val_loss: 0.7845 - val_output_1_loss: 0.0069 - val_output_2_loss: 0.3164 - val_output_3_loss: 0.3701 - val_output_4_loss: 0.0615
Epoch 107/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.7420 - output_1_loss: 0.0044 - output_2_loss: 0.3100 - output_3_loss: 0.3869 - output_4_loss: 0.0141 - val_loss: 0.7934 - val_output_1_loss: 0.0070 - val_output_2_loss: 0.3134 - val_output_3_loss: 0.3695 - val_output_4_loss: 0.0737
Epoch 108/600
4798/4798 [===========================

...

Epoch 495/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.5133 - output_1_loss: 0.0036 - output_2_loss: 0.1993 - output_3_loss: 0.2607 - output_4_loss: 0.0139 - val_loss: 0.6064 - val_output_1_loss: 0.0058 - val_output_2_loss: 0.2010 - val_output_3_loss: 0.3001 - val_output_4_loss: 0.0615
Epoch 496/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.5539 - output_1_loss: 0.0038 - output_2_loss: 0.1990 - output_3_loss: 0.2904 - output_4_loss: 0.0128 - val_loss: 0.8935 - val_output_1_loss: 0.0070 - val_output_2_loss: 0.2283 - val_output_3_loss: 0.4834 - val_output_4_loss: 0.0482
Epoch 497/600
4798/4798 [==============================] - 0s 85us/sample - loss: 0.6625 - output_1_loss: 0.0043 - output_2_loss: 0.2035 - output_3_loss: 0.3476 - output_4_loss: 0.0123 - val_loss: 0.6624 - val_output_1_loss: 0.0058 - val_output_2_loss: 0.2079 - val_output_3_loss: 0.3330 - val_output_4_loss: 0.0490
Epoch 498/600
4798/4798 [===========================

Epoch 521/600
4798/4798 [==============================] - 0s 86us/sample - loss: 0.5457 - output_1_loss: 0.0033 - output_2_loss: 0.1937 - output_3_loss: 0.2820 - output_4_loss: 0.0126 - val_loss: 0.5703 - val_output_1_loss: 0.0049 - val_output_2_loss: 0.1925 - val_output_3_loss: 0.2710 - val_output_4_loss: 0.0516
Epoch 522/600
4798/4798 [==============================] - 0s 102us/sample - loss: 0.4976 - output_1_loss: 0.0034 - output_2_loss: 0.1898 - output_3_loss: 0.2537 - output_4_loss: 0.0131 - val_loss: 0.5304 - val_output_1_loss: 0.0052 - val_output_2_loss: 0.1913 - val_output_3_loss: 0.2469 - val_output_4_loss: 0.0538
Epoch 523/600
4798/4798 [==============================] - 0s 87us/sample - loss: 0.4779 - output_1_loss: 0.0033 - output_2_loss: 0.1895 - output_3_loss: 0.2438 - output_4_loss: 0.0124 - val_loss: 0.5876 - val_output_1_loss: 0.0059 - val_output_2_loss: 0.1951 - val_output_3_loss: 0.2935 - val_output_4_loss: 0.0561
Epoch 524/600
4798/4798 [==========================

Epoch 547/600
4798/4798 [==============================] - 0s 89us/sample - loss: 1.8249 - output_1_loss: 0.0072 - output_2_loss: 0.7212 - output_3_loss: 0.9470 - output_4_loss: 0.0157 - val_loss: 1.9564 - val_output_1_loss: 0.0090 - val_output_2_loss: 0.7175 - val_output_3_loss: 1.0255 - val_output_4_loss: 0.0449
Epoch 548/600
4798/4798 [==============================] - 0s 92us/sample - loss: 1.7874 - output_1_loss: 0.0068 - output_2_loss: 0.7113 - output_3_loss: 0.9269 - output_4_loss: 0.0154 - val_loss: 1.9261 - val_output_1_loss: 0.0081 - val_output_2_loss: 0.7087 - val_output_3_loss: 1.0118 - val_output_4_loss: 0.0451
Epoch 549/600
4798/4798 [==============================] - 0s 89us/sample - loss: 1.7567 - output_1_loss: 0.0068 - output_2_loss: 0.7012 - output_3_loss: 0.9117 - output_4_loss: 0.0163 - val_loss: 1.8900 - val_output_1_loss: 0.0088 - val_output_2_loss: 0.6985 - val_output_3_loss: 0.9859 - val_output_4_loss: 0.0555
Epoch 550/600
4798/4798 [===========================

Epoch 573/600
4798/4798 [==============================] - 0s 87us/sample - loss: 0.9076 - output_1_loss: 0.0062 - output_2_loss: 0.3400 - output_3_loss: 0.4858 - output_4_loss: 0.0165 - val_loss: 0.9478 - val_output_1_loss: 0.0076 - val_output_2_loss: 0.3394 - val_output_3_loss: 0.4910 - val_output_4_loss: 0.0481
Epoch 574/600
4798/4798 [==============================] - 0s 84us/sample - loss: 0.8954 - output_1_loss: 0.0057 - output_2_loss: 0.3313 - output_3_loss: 0.4829 - output_4_loss: 0.0155 - val_loss: 0.9608 - val_output_1_loss: 0.0075 - val_output_2_loss: 0.3305 - val_output_3_loss: 0.5012 - val_output_4_loss: 0.0598
Epoch 575/600
4798/4798 [==============================] - 0s 85us/sample - loss: 0.8833 - output_1_loss: 0.0056 - output_2_loss: 0.3232 - output_3_loss: 0.4759 - output_4_loss: 0.0144 - val_loss: 0.9558 - val_output_1_loss: 0.0096 - val_output_2_loss: 0.3367 - val_output_3_loss: 0.4898 - val_output_4_loss: 0.0528
Epoch 576/600
4798/4798 [===========================

Epoch 599/600
4798/4798 [==============================] - 0s 85us/sample - loss: 0.9387 - output_1_loss: 0.0074 - output_2_loss: 0.3635 - output_3_loss: 0.4769 - output_4_loss: 0.0148 - val_loss: 1.0070 - val_output_1_loss: 0.0085 - val_output_2_loss: 0.3708 - val_output_3_loss: 0.5061 - val_output_4_loss: 0.0444
Epoch 600/600
4798/4798 [==============================] - 0s 84us/sample - loss: 0.9107 - output_1_loss: 0.0066 - output_2_loss: 0.3577 - output_3_loss: 0.4640 - output_4_loss: 0.0123 - val_loss: 0.9780 - val_output_1_loss: 0.0084 - val_output_2_loss: 0.3657 - val_output_3_loss: 0.4873 - val_output_4_loss: 0.0455


In [4]:
# Look at learning rates vs AEC-only and full-model losses
lrs = []
full_losses = []
aec_losses = []
for i in range(num_init_models):
    _, full_hist, lr, aec_hist = models[i]
    full_losses.append(np.mean(full_hist.history['loss'][-3:]))
    aec_losses.append(np.mean(aec_hist.history['loss'][-3:]))
    lrs.append(lr)


## Doubled down on JSON for saving the data, since it is a uniform format!!

# Get the dictionary containing each metric and the loss for each epoch
history_dict = hist.history.copy()

# For some reason, float32s cannot be serialized... so the training losses are
# converted to float64s
for key in history_dict:
    history_dict[key] = [val.astype(np.float64) for val in history_dict[key]]
    
# And now dump it
hist_filepath = "./results/{}_model_history.json".format(expt_name)
json.dump(history_dict, open(hist_filepath, 'w'))

# Also dump the full_losses, aec_losses, and learning rates
initial_training = {'aec_only_loss': aec_losses,
                    'full_init_loss': full_losses,
                    'learn_rates': lrs,
                    'best_lr': best_lr}
init_train_filepath = "./results/{}_initial_train.json".format(expt_name)
json.dump(initial_training, open(init_train_filepath, 'w'))